# Predicting Movie Rental Durations
## predict -> how many days a customer will rent a DVD
This project aims to help a DVD rental company predict how many days a customer will rent a DVD. The objective is to build a regression model that achieves a Mean Squared Error (MSE) of 3 or less. Such a model will assist the company in optimizing inventory management and rental planning.

In [102]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.cm as cm
from sklearn import preprocessing
import xgboost
import seaborn as sb
from sklearn.model_selection import train_test_split

In [112]:
mv = pd.read_csv('C:/Users/LENOVO/Desktop/rental_info.csv')
mv.head(100)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2005-07-06 17:07:30+00:00,2005-07-09 20:26:30+00:00,0.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,0.9801,15129.0,0.9801
96,2005-07-28 05:16:51+00:00,2005-08-02 04:45:51+00:00,1.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,3.9601,15129.0,0.9801
97,2005-08-23 02:49:23+00:00,2005-09-01 07:50:23+00:00,6.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,48.8601,15129.0,0.9801
98,2005-05-29 01:23:02+00:00,2005-06-02 21:39:02+00:00,1.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,3.9601,15129.0,0.9801


In [86]:
mv.describe()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100


In [126]:
mv["rental_length"] = (pd.to_datetime(mv["return_date"]) - pd.to_datetime(mv["rental_date"]) ).dt.days
mv["deleted_scenes"] = np.where(mv["special_features"].str.contains("Deleted Scenes"),1,0)
mv["behind_the_scenes"] = np.where(mv["special_features"].str.contains("Behind the Scenes"),1,0)

In [128]:
z = mv.columns
z

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'rental_length',
       'deleted_scenes', 'behind_the_scenes'],
      dtype='object')

In [130]:
X = mv[[ 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost','deleted_scenes', 'behind_the_scenes', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'rental_length']]
y = mv["rental_length"] 

In [174]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =0)

In [176]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score
rf = RandomForestRegressor(random_state=9)
param_dist = {'n_estimators': np.arange(1, 101, 1), 'max_depth': np.arange(1, 11, 1)}
rand_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, random_state=9)
rand_search.fit(X_train, y_train)
best_params = rand_search.best_params_
rf_best = RandomForestRegressor(n_estimators=best_params["n_estimators"],max_depth=best_params["max_depth"],random_state=9)
rf_best.fit(X_train, y_train)
yhat = rf_best.predict(X_test)
print(f"Sklearn RF Accuracy: {accuracy_score(y_test, yhat):.3f}")
print(f"Sklearn RF F1-Score: {f1_score(y_test, yhat, average='weighted'):.3f}")

Sklearn RF Accuracy: 1.000
Sklearn RF F1-Score: 1.000
